In [1]:
%matplotlib inline
import pandas as pd

In [2]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime
from pyspark import SQLContext
sqlContext = SQLContext(sc)

import pyspark.sql.functions as F
import numpy as np

%matplotlib inline
import matplotlib
matplotlib.use('Agg')
#import matplotlib.pyplot
import matplotlib.pyplot as plt

/Users/l744679/Applications/anaconda/lib/python2.7/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)
/Users/l744679/Applications/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Installed autotime.py. To use it, type:
  %load_ext autotime


In [2]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [3]:
titles = pd.DataFrame.from_csv('data/titles.csv', index_col=None)
titles.head()

,title,year
0,A Lélek órása,1923
1,Aizaugusa gravi viegli krist,1986
2,Agliyorum,1988
3,0_1_0,2008
4,97 fung lau mung,1994


In [4]:
cast = pd.DataFrame.from_csv('data/cast.csv', index_col=None)
cast.head()

,title,year,name,type,character,n
0,The Core,2003,Alejandro Abellan,actor,U.S.S. Soldier,NaN
1,Il momento di uccidere,1968,Remo De Angelis,actor,Dago,9
2,Across the Divide,1921,Thomas Delmar,actor,Dago,4
3,Revan,2012,Diego James,actor,Dago,NaN
4,Un homme marche dans la ville,1950,Fabien Loris,actor,Dago,12


In [6]:
titles = sqlContext.read.parquet(os.path.expanduser('~/Dropbox/Projects/data/movielens_p/titles'))
titles.cache()
cast = sqlContext.read.parquet(os.path.expanduser('~/Dropbox/Projects/data/movielens_p/cast'))
cast.cache()

DataFrame[title: string, year: int, name: string, type: string, character: string, n: int]

time: 276 ms


### Using groupby(), plot the number of films that have been released each decade in the history of cinema.

In [9]:
titles.withColumn('decade', F.floor(titles.year / 10) * 10).groupby('decade').count().sort('decade').show()

+------+-----+
|decade|count|
+------+-----+
|  1890|    6|
|  1900|   35|
|  1910| 6337|
|  1920| 8648|
|  1930| 9954|
|  1940| 8301|
|  1950|12051|
|  1960|16421|
|  1970|17704|
|  1980|19257|
|  1990|21251|
|  2000|37659|
|  2010|55129|
|  2020|   37|
+------+-----+

time: 467 ms


### Use groupby() to plot the number of "Hamlet" films made each decade.

In [10]:
titles[titles.title == 'Hamlet'].withColumn('decade', F.floor(titles.year / 10) * 10).groupby('decade').count().sort('decade').show()

+------+-----+
|decade|count|
+------+-----+
|  1910|    3|
|  1920|    1|
|  1940|    1|
|  1950|    1|
|  1960|    2|
|  1970|    2|
|  1980|    1|
|  1990|    2|
|  2000|    2|
|  2010|    4|
+------+-----+

time: 416 ms


### How many leading (n=1) roles were available to actors, and how many to actresses, in each year of the 1950s?

In [13]:
cast[(F.floor(cast.year / 10) == 195) & (cast.n == 1)].groupby(['year', 'type']).count().sort(['year', 'type']).show()

+----+-------+-----+
|year|   type|count|
+----+-------+-----+
|1950|  actor|  604|
|1950|actress|  268|
|1951|  actor|  632|
|1951|actress|  272|
|1952|  actor|  589|
|1952|actress|  285|
|1953|  actor|  632|
|1953|actress|  289|
|1954|  actor|  623|
|1954|actress|  297|
|1955|  actor|  606|
|1955|actress|  264|
|1956|  actor|  608|
|1956|actress|  288|
|1957|  actor|  706|
|1957|actress|  283|
|1958|  actor|  691|
|1958|actress|  275|
|1959|  actor|  676|
|1959|actress|  285|
+----+-------+-----+

time: 718 ms


### In the 1950s decade taken as a whole, how many total roles were available to actors, and how many to actresses, for each "n" number 1 through 5?

In [15]:
cast[(F.floor(cast.year / 10) == 195) & (cast.n.cast('int') < 6)].groupby(['n', 'type']).count().sort(['n', 'type']).show()

+---+-------+-----+
|  n|   type|count|
+---+-------+-----+
|  1|  actor| 6367|
|  1|actress| 2806|
|  2|  actor| 4371|
|  2|actress| 4391|
|  3|  actor| 5394|
|  3|actress| 3011|
|  4|  actor| 5391|
|  4|actress| 2740|
|  5|  actor| 5420|
|  5|actress| 2439|
+---+-------+-----+

time: 559 ms


### Use groupby() to determine how many roles are listed for each of the Pink Panther movies.

In [21]:
cast[cast.title == 'The Pink Panther'].show()

+----------------+----+--------------------+-----+--------------------+----+
|           title|year|                name| type|           character|   n|
+----------------+----+--------------------+-----+--------------------+----+
|The Pink Panther|2006|      William Abadie|actor|                Bizu|  11|
|The Pink Panther|2006|   Scott (II) Adkins|actor|            Jacquard|  20|
|The Pink Panther|2006|Patrick (III) And...|actor|     NYC Firefighter|null|
|The Pink Panther|2006|      Michael Arthur|actor|          Officer #1|null|
|The Pink Panther|1963|         John Bartha|actor|           Policeman|null|
|The Pink Panther|2006|         Dexter Bell|actor|   Terry Ahkee-Sauce|  33|
|The Pink Panther|1963|      Riccardo Billi|actor|   Aristotle Sarajos|  11|
|The Pink Panther|2006| Antoine Blanquefort|actor|   Ballroom Reporter|  44|
|The Pink Panther|2006|      Davide Borella|actor|     Gas-Mask Bandit|  50|
|The Pink Panther|2006|   David (IV) Boston|actor|    Blackjack Dealer|null|

In [20]:
cast[cast.title == 'The Pink Panther'].groupby('year').max('n').show()

+----+------+
|year|max(n)|
+----+------+
|1963|    15|
|2006|    50|
+----+------+

time: 515 ms


### Use groupby() to determine how many roles are listed for each of movies with Pink Panther as a character.

In [17]:
cast[cast.character == 'Pink Panther'].show()

+----------------+----+-----------+-----+------------+----+
|           title|year|       name| type|   character|   n|
+----------------+----+-----------+-----+------------+----+
|Long teng hu yue|1983|Li-Peng Wan|actor|Pink Panther|null|
| Xin tiao yi bai|1987| Ching Wong|actor|Pink Panther|null|
+----------------+----+-----------+-----+------------+----+

time: 279 ms


In [24]:
cast.join(cast[cast.character == 'Pink Panther'].select('title', 'year').distinct(), ['title', 'year']).show()

+----------------+----+--------------+-------+--------------------+----+
|           title|year|          name|   type|           character|   n|
+----------------+----+--------------+-------+--------------------+----+
| Xin tiao yi bai|1987|    Kent Cheng|  actor|            Sgt. Chu|null|
| Xin tiao yi bai|1987|    Mark Cheng|  actor|          Weeny Eyes|null|
| Xin tiao yi bai|1987|      Fong Liu|  actor|              To Nam|null|
| Xin tiao yi bai|1987|  Fui-On Shing|  actor|                 Hai|null|
| Xin tiao yi bai|1987|    Ching Wong|  actor|        Pink Panther|null|
| Xin tiao yi bai|1987|      Fung Woo|  actor|           Uncle Yam|null|
| Xin tiao yi bai|1987| Maggie Cheung|actress|              Maggie|null|
| Xin tiao yi bai|1987|    Bonnie Law|actress|           Chu Chung|null|
|Long teng hu yue|1983|   Jackie Chan|  actor|           Chan Lung|   1|
|Long teng hu yue|1983|  Hui Lou Chen|  actor|        Chan Chi-Pei|   8|
|Long teng hu yue|1983|Yeong-Mun Kwon|  actor|     

In [25]:
cast.join(cast[cast.character == 'Pink Panther'].select('title', 'year').distinct(), ['title', 'year']).groupby('year').max('n').show()

+----+------+
|year|max(n)|
+----+------+
|1983|     8|
|1987|  null|
+----+------+

time: 2.28 s


### List, in order by year, each of the films in which Frank Oz has played more than 1 role.

In [26]:
cast[cast.name == 'Frank Oz'].show()

+--------------------+----+--------+-----+--------------------+---+
|               title|year|    name| type|           character|  n|
+--------------------+----+--------+-----+--------------------+---+
|An American Werew...|1981|Frank Oz|actor|         Mr. Collins| 13|
|An American Werew...|1981|Frank Oz|actor|          Miss Piggy| 13|
| Blues Brothers 2000|1998|Frank Oz|actor|              Warden|  4|
|      Innocent Blood|1992|Frank Oz|actor|         Pathologist| 21|
|          Inside Out|2015|Frank Oz|actor|Subconscious Guar...| 14|
|           Labyrinth|1986|Frank Oz|actor|         The Wiseman| 17|
|      Monsters, Inc.|2001|Frank Oz|actor|              Fungus|  9|
|Muppet Treasure I...|1996|Frank Oz|actor|          Miss Piggy| 14|
|Muppet Treasure I...|1996|Frank Oz|actor|         Fozzie Bear| 14|
|Muppet Treasure I...|1996|Frank Oz|actor|       Sam the Eagle| 14|
|Muppet Treasure I...|1996|Frank Oz|actor|              Animal| 14|
|  Muppets from Space|1999|Frank Oz|actor|      

In [30]:
cast[cast.name == 'Frank Oz'].groupby(['title', 'year']).count().filter('count > 1').sort('year').show(truncate=False)

+----------------------------------------+----+-----+
|title                                   |year|count|
+----------------------------------------+----+-----+
|The Muppet Movie                        |1979|8    |
|The Great Muppet Caper                  |1981|6    |
|An American Werewolf in London          |1981|2    |
|The Dark Crystal                        |1982|2    |
|The Muppets Take Manhattan              |1984|7    |
|Sesame Street Presents: Follow that Bird|1985|3    |
|The Muppet Christmas Carol              |1992|7    |
|Muppet Treasure Island                  |1996|4    |
|Muppets from Space                      |1999|4    |
|The Adventures of Elmo in Grouchland    |1999|3    |
+----------------------------------------+----+-----+

time: 488 ms


### List each of the characters that Frank Oz has portrayed at least twice.

In [33]:
cast[cast.name == 'Frank Oz'].groupby(['character']).count().filter('count > 2').sort('count', ascending=False).show(truncate=False)

+--------------+-----+
|character     |count|
+--------------+-----+
|Miss Piggy    |6    |
|Animal        |6    |
|Sam the Eagle |5    |
|Yoda          |5    |
|Fozzie Bear   |4    |
|Bert          |3    |
|Cookie Monster|3    |
+--------------+-----+

time: 371 ms
